In [169]:
### Import libraries
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn import cross_validation
import matplotlib.pyplot as plt
%pylab
from sklearn.pipeline import make_pipeline
from scipy.signal import argrelextrema

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [170]:
df_satData = pd.read_csv('satData_woOutliers.csv')

In [171]:
cols = df_satData.columns.tolist()

In [172]:
df_labels =  df_satData[['101']+cols[cols.index('MSL_1'):]]

In [173]:
df_dev_gr = pd.read_excel('dev_gr_labels_slack.xlsx')
cols = df_dev_gr.columns.tolist()
df_dev_gr = df_dev_gr[['censuscode']+cols[-4:]]

for col in cols[-4:]:
    df_temp = pd.get_dummies(df_dev_gr[col])
    temp_cols = [col+'_g'+str(i) for i in range(len(list(df_temp)))]
    df_temp.columns = temp_cols

    df_dev_gr =pd.concat([df_dev_gr,df_temp],axis=1)

In [174]:
cols= df_dev_gr.columns.tolist()

In [175]:
df_labels = df_labels.merge(df_dev_gr,left_on='101',right_on='censuscode')

In [176]:
cols = df_labels.columns.tolist()

In [177]:
xvar = cols[1:cols.index('censuscode')]

In [178]:
yvar = cols[cols.index('censuscode')+5:]

In [179]:
X = df_labels[xvar].values

In [180]:
model = LogisticRegression()

In [181]:
np.shape(X)

(574, 21)

In [182]:
yvar2 =  [u'Sub1',
 u'Sub2',
 u'Sub3',
 u'Sub4']

In [183]:
yvar2

[u'Sub1', u'Sub2', u'Sub3', u'Sub4']

In [184]:
df_labels['growth'] = (df_labels[yvar2]).max(axis=1).values

In [186]:
df_growth = pd.get_dummies(df_labels['growth'])

In [187]:
df_growth.columns = ['g1','g2','g3']

In [190]:
df_labels = pd.concat([df_labels,df_growth],axis=1)

In [192]:
yvar3 = ['g1','g2','g3']

In [198]:
df_score = pd.DataFrame(columns=['f1','acc','numDist'])
for label in yvar3:
    y = df_labels[label].values
    model = LogisticRegression(class_weight='balanced')
    y_pred = cross_validation.cross_val_predict(model,X,y, cv=5)
    
    f1 = metrics.f1_score(y,y_pred)
    acc = metrics.accuracy_score(y,y_pred)
    count_1 = len(df_labels[df_labels[label] == 1])
    
    df_score.loc[label] = [f1,acc,count_1]

In [199]:
df_score

,f1,acc,numDist
g1,0.448649,0.644599,139.0
g2,0.677966,0.635889,333.0
g3,0.478114,0.729965,102.0


In [196]:
df_score = pd.DataFrame(columns=['f1','acc','numDist'])
for label in yvar:
    y = df_labels[label].values
    model = LogisticRegression(class_weight='balanced')
    y_pred = cross_validation.cross_val_predict(model,X,y, cv=5)
    
    f1 = metrics.f1_score(y,y_pred)
    acc = metrics.accuracy_score(y,y_pred)
    count_1 = len(df_labels[df_labels[label] == 1])
    
    df_score.loc[label] = [f1,acc,count_1]

In [138]:
df_score.sort_values(by='f1',ascending=False)

,f1,acc,numDist
Sub4_g0,0.927757,0.867596,562.0
Sub3_g0,0.867368,0.780488,529.0
Sub2_g0,0.749004,0.670732,439.0
Sub1_g0,0.700240,0.782230,192.0
Sub1_g2,0.680147,0.696864,262.0
Sub2_g3,0.382353,0.853659,33.0
Sub1_g3,0.314961,0.696864,60.0
Sub2_g2,0.284585,0.684669,57.0
Sub3_g1,0.250000,0.790941,25.0
Sub2_g1,0.244094,0.665505,45.0


## Building features from night light data

In [ ]:
var = []
for i in range(64):
    var.append('DN_' + str(i).zfill(2))

df_satData['Area'] = 0
for i in var:
    df_satData['Area']=df_satData['Area'] + df_satData[i]
df_satData['Area'] = df_satData['Area'].replace(0,1)

for i in var:
    df_satData['p'+i] = (df_satData[i])/(df_satData['Area']*1.0)

for i,j in enumerate(var):
    df_satData['q'+j] = df_satData[j]*(i+1)

var3 = []
for i in var:
    var3.append('p'+i)
for i in var:
    var3.append('q'+i)

In [ ]:
df_satData[var3]

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
sse = {}
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, max_iter=1000).fit(df_satData[var3])
    #print(data["clusters"])
    sse[k] = kmeans.inertia_ # Inertia: Sum of distances of samples to their closest cluster center
plt.figure()
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Number of cluster")
plt.ylabel("SSE")
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=3, max_iter=1000).fit(df_satData[var3].T)
df_ntlBins = pd.DataFrame(kmeans.labels_)

## Appling Pca to matrix formed from features of nightlight data

In [ ]:
X = df_satData.loc[:,var3].values

In [ ]:
X = preprocessing.scale(X)
pca = PCA()
pca.fit(X)

#pca= PCA(svd_solver='full')
#pca.fit(X, y=None )

#The amount of variance that each PC explains
var= pca.explained_variance_ratio_

#Cumulative Variance explains
var1=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)
print(var1)

#Number of PC comprising 95%Variance
n = len(var1)-len([num for num in var1 if num != 0 and num > 85.0])+1
print( "n: 95% Variance: ",n)

In [ ]:
### PCA fit
X=pca.fit_transform(X)
X=X[:,0:n]

In [ ]:
n

In [ ]:
## creating variance table 

In [201]:
df_pcaComp = pd.DataFrame(pca.components_)

NameError: name 'pca' is not defined

In [202]:
pca_idx = ['PCA_'+str(i) for i in range(len(df_pcaComp))]

NameError: name 'df_pcaComp' is not defined

In [203]:
df_pcaComp.index = pca_idx

NameError: name 'pca_idx' is not defined

In [ ]:
df_pcaComp.columns = var3

In [ ]:
df_pcaComp[:9].to_csv('pcaVarTableRaw.csv')

In [200]:
bins_pcaVar = [[0]*10]*7

for column in df_pcaComp:
    bins_pcaVar[(int(column[4:])/10)] += ((df_pcaComp[column][:10])/abs(df_pcaComp[:10]).sum(axis=1))

bins_pcaVar[5] += bins_pcaVar[6]

NameError: name 'df_pcaComp' is not defined

In [ ]:
df_bins_pcaVar = pd.DataFrame(bins_pcaVar[:6]).T

In [ ]:
df_bins_pcaVar.columns = ['L1','L2','M1','M2','H1','H2']

In [ ]:
df_bins_pcaVar

In [ ]:
df_newVarTable = pd.DataFrame(columns=['0','1','2','3','4','5'])

for i in range(10):
    temp = df_bins_pcaVar[i:i+1].sort_values(by=pca_idx[i],axis=1,ascending=False)
    temp_cols = temp.columns.tolist()
    temp_vals = np.round(temp.values[0],2).tolist()
    new_row = [str(temp_vals[j])+','+temp_cols[j] for j in range(len(temp_cols))]
    df_newVarTable.loc[pca_idx[i],:] = new_row
    

In [ ]:
#df_newVarTable.to_csv('newVarTable.csv')

In [ ]:
### Creating Dataframe of PCA
col_X = []
for j in range(n):
    col_X.append('PCA_'+str(j))
df_pcaNTL = pd.DataFrame(X,columns=col_X)

In [ ]:
df_pcaNTL['101'] = df_satData['101'].values.tolist()

In [ ]:
df_pcaNTL.head()

## Building features out of Modis Data

In [ ]:
df_satData['Urban and built-up']=df_satData['Urban and built-up']+1
df_satData['CropRatio']=(df_satData['Croplands']+df_satData['Cropland/Natural vegetation mosaic'])/df_satData['Area']
df_satData['AvgUrbanNTL']=df_satData['sum']/df_satData['Urban and built-up']
df_satData['UrbanRatio']= df_satData['Urban and built-up']/df_satData['Area']

df_satData['Natural']= 0
for i in ['Water','Evergreen Needleleaf forest','Evergreen Broadleaf forest','Deciduous Needleleaf forest','Deciduous Broadleaf forest','Mixed forest','Closed shrublands','Open shrublands','Woody savannas','Savannas','Grasslands','Permanent wetlands','Snow and ice']:
    df_satData['Natural']=df_satData['Natural']+df_satData[i]
    
df_satData['CropRemainRatio']=(df_satData['Croplands']+df_satData['Cropland/Natural vegetation mosaic'])/(df_satData['Area']-df_satData['Natural'])
df_satData['UrbanRemainRatio']=df_satData['Urban and built-up']/(df_satData['Area']-df_satData['Natural'])
#df_satData['UrbanRatio']=df_satData['UrbanRatio'].fillna(0)


forest =['Evergreen Broadleaf forest','Deciduous Broadleaf forest','Mixed forest']
df_satData['forest'] = df_satData[forest].sum(axis=1)

grass_shrubs =['Closed shrublands','Open shrublands','Woody savannas','Savannas','Grasslands']
df_satData['grass_shrubs'] = df_satData[grass_shrubs].sum(axis=1)


In [ ]:
modi_area = df_satData[pure_modis_features].sum(axis=1).values.tolist()

In [ ]:
left_area = [df_satData['Area'].values.tolist()[i]-modi_area[i] for i in range(len(df_satData))]

In [ ]:
df_satData['left_area'] = left_area

In [ ]:
modi_var = []
modi_var=['mean','sum','CropRatio', 'AvgUrbanNTL', 'UrbanRatio', 'CropRemainRatio', 'UrbanRemainRatio','Natural']
for i in ['Croplands',
 'Urban and built-up',
 'Cropland/Natural vegetation mosaic',
 'forest',
 'grass_shrubs']:
    df_satData['mod_'+i]=df_satData[i]/df_satData['Area']
    modi_var.append('mod_'+i)
    modi_var.append(i)
#modi_var.append('left_area')

## final dataframe from pca version of nl and modi 

In [ ]:
## only nighlight
df_modi_nlPca =  pd.merge(df_satData[['101','mean','sum']],df_pcaNTL,left_on='101',right_on='101',how='inner')

In [ ]:
df_modi_nlPca =  pd.merge(df_satData[['101']+modi_var],df_pcaNTL,left_on='101',right_on='101',how='inner')

In [ ]:
## for modis only
df_modi_nlPca = df_satData[['101'] + modi_var]

In [ ]:
df_modi_nlPca.shape

In [ ]:
var = df_modi_nlPca.columns.tolist()[5:]

In [ ]:
var

In [ ]:
var.extend(['CropRatio'])

In [ ]:
var

## finding feature ranking for a particular label

In [ ]:
def featureRank(label):
    model = LogisticRegression()
    selector = RFE(model,1, step=1)

    X = df_modi_nlPca[var].values
    X = preprocessing.scale(X)

    y = df_satData.loc[:,label].values

    selector.fit(X,y)

    selectedIdx = selector.get_support(indices=True)

    featureRanking = selector.ranking_

    ranked_var = pd.DataFrame(featureRanking,var)
    ranked_var = ranked_var.reset_index()
    ranked_var.columns = (['var','rank'])
    ranked_var = ranked_var.sort_values('rank')
    ranked_var.reset_index(inplace=True)
    del ranked_var['index']
    
    return ranked_var

## selecting features to maximize f1_score
#### Assumption: Set of first n features in feature ranking will produce maximum f1_score compared to any other set of n features.

In [ ]:
def selectFeatures(ranked_var,label):
    metric_list = list()
    f1_list = list()
    for i in range(len(ranked_var)):
        var2 = ranked_var.loc[:i,'var'].values.tolist()

        X = df_modi_nlPca.loc[:,var2].values
        y = df_satData.loc[:,label].values

        model = LogisticRegression(class_weight='balanced')
        clf = make_pipeline(preprocessing.StandardScaler(),model)
        predicted = cross_validation.cross_val_predict(clf,X,y, cv=5)

        acc = metrics.accuracy_score(y,predicted)
        recall = metrics.recall_score(y,predicted)
        prec =  metrics.precision_score(y,predicted)

        f1 = metrics.f1_score(y,predicted)

        metric_list.append([f1,acc,recall,prec])
        f1_list.append(f1)

    df_metric = pd.DataFrame(metric_list)
    df_metric.columns =  (['f1','acc','recall','prec'])

    ## finding the global maximum index in f1_list
    global_maxima = f1_list.index(max(f1_list))

    ## plotting f1_score with features selected
    #figure()
    #plt.grid()
    #plt.plot(f1_list)
    #plt.title(label)
    #plt.xlabel('number of variables')
    #plt.ylabel('f1_score')
    
    ## findi  ng indexes of all local maximums

    x = array(f1_list)
    # for local maxima
    local_maxima = argrelextrema(x, np.greater)[0].tolist()

    f1_acceptable = f1_list[global_maxima]-0.01

    i = 0
    while f1_list[i] < f1_acceptable:
        i += 1

    var_final = ranked_var.loc[:i,'var']

    X = df_modi_nlPca.loc[:,var_final].values
    y = df_satData.loc[:,label].values

    model = LogisticRegression()

    fit = model.fit(X,y)

    df_selecedVar = ranked_var.loc[:i].copy(deep=True)
    df_selecedVar.insert(2,'coeff',fit.coef_[0])

    return [df_selecedVar,f1_list[i],df_metric[i:i+1]]

In [ ]:
df_overallScore = pd.DataFrame(columns=['f1','acc'])
for main_label in ['MSL','MSW','CHH','FC','BF','EMP']:
    lis_dis_label = [main_label+'_'+str(i) for i in range(1,4)]
    yTrue = []
    yPred = []
    for label in lis_dis_label:
        ranked_var = featureRank(label)
        feature_and_score = selectFeatures(ranked_var,label)
        var2 = feature_and_score[0]['var'].values.tolist()
        X = df_modi_nlPca.loc[:,var2].values
        y = df_satData.loc[:,label].values
        yTrue.extend(y)
        
        model = LogisticRegression(class_weight='balanced')
        clf = make_pipeline(preprocessing.StandardScaler(),model)
        predicted = cross_validation.cross_val_predict(clf,X,y, cv=5)
        yPred.extend(predicted)
    acc = metrics.accuracy_score(yTrue,yPred)
    f1 = metrics.f1_score(yTrue,yPred)  
    
    df_overallScore.loc[main_label] = [f1,acc]

In [ ]:
np.round(df_overallScore.sort_values(by='f1',ascending=False),2)#.to_csv('overallScore.csv')

In [ ]:
label = lis_dis_label[0]

In [ ]:
metric_list = list()
f1_list = list()
for i in range(len(ranked_var)):
    var2 = ranked_var.loc[:i,'var'].values.tolist()

    X = df_modi_nlPca.loc[:,var2].values
    y = df_satData.loc[:,label].values

    model = LogisticRegression(class_weight='balanced')
    clf = make_pipeline(preprocessing.StandardScaler(),model)
    predicted = cross_validation.cross_val_predict(clf,X,y, cv=5)


In [ ]:
for label in lis_dis_label:
    ranked_var = featureRank(label)
    feature_and_score = selectFeatures(ranked_var,label)

In [ ]:
var_temp = feature_and_score[0]['var'].values.tolist()

In [ ]:
select_feature = []
score_metric = []
for label in lis_labelsAll:
    ranked_var = featureRank(label)
    feature_and_score = selectFeatures(ranked_var,label)
    select_feature.append(feature_and_score)
    score_metric.append(feature_and_score[2].values[0])

In [ ]:
df_scoreMetric = pd.DataFrame(score_metric,columns = ['f1','acc','recall','prec'], index=lis_labelsAll)

In [ ]:
#df_scoreMetric.to_csv('scoreMetric_asset.csv')

In [ ]:
np.round(df_scoreMetric.sort_values(by='f1',ascending=False),2)

In [ ]:
select_feature[0][0]

In [ ]:
## adding pvalues.... to the list of featues.

In [ ]:
X = df_modi_nlPca.copy(deep=True)
X.insert(1,'intercept',1.0)
cols = X.columns.tolist()

In [ ]:
modi_cols = cols[1:cols.index('PCA_0')]
pca_cols = ['intercept'] + cols[cols.index('PCA_0'):]

In [ ]:
def find_summary(cols,label):
    X1 = X[cols].values
    y = df_satData.loc[:,label].values

    model = sm.Logit(y,X1)
    result = model.fit()
    summary = result.summary()

    df_summary = pd.DataFrame(result.params,result.pvalues)
    df_summary.reset_index(inplace=True)
    df_summary.index = cols
    df_summary.columns = ['pvals','param']
    
    return df_summary

In [ ]:
selectedFeatures_pval = []
for i,label in enumerate(lis_labelsAll):
    df_temp = select_feature[i][0].copy(deep=True)
    df_temp.index = df_temp['var']
    del df_temp['var']
    df_summary = pd.concat([find_summary(modi_cols,label),find_summary(pca_cols,label)])
    df_temp2 = pd.concat([df_temp,df_summary],axis=1,join_axes=[df_temp.index])
    selectedFeatures_pval.append(df_temp2)

In [ ]:
label = 'BF_1'
selectedFeatures_pval[lis_labelsAll.index(label)]